iterate all files related to CAPS



In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import date 

store each into one big df


In [22]:
import os

#https://stackoverflow.com/a/21232849 model 
def getCAPSByDateAndType(type):  #returns a dict, date + df caps for that date, then extended date and time
                                # print("for type, ", type)
    ret = []
    for root, dirs, files in os.walk("."):
        for filename in files:
            if type in filename:
                # print("CAPS file, ", filename)
                ret.append(filename)
    return ret

arr = getCAPSByDateAndType("CAPS")
li = [] 
for filename in arr:
    csv = "lob_caps/" + filename
    print(csv)
    df = pd.read_csv(csv, index_col=None, header=0)
    li.append(df)

endFrame = pd.concat(li, axis=0, ignore_index=True)
endFrame.sort_values(by=['time'], ascending=True)
print("for new df: ", endFrame.shape[0])
start = endFrame["time"].min()
end = endFrame["time"].max()
print("start: ", start, " end: ", end)

lob_caps/Fri Aug 26 2022 06:27:17 GMT-0700 (Pacific Daylight Time)-CAPS.csv
lob_caps/Sat Oct 01 2022 08:39:01 GMT-0700 (Pacific Daylight Time)-CAPS.csv
lob_caps/Fri Dec 02 2022 09:40:57 GMT-0800 (Pacific Standard Time)-CAPS.csv
lob_caps/Thu Aug 25 2022 10:42:37 GMT-0700 (Pacific Daylight Time)-CAPS.csv
lob_caps/Mon Oct 31 2022 17:15:31 GMT-0700 (Pacific Daylight Time)-CAPS.csv
lob_caps/Tue Nov 08 2022 22:29:16 GMT-0800 (Pacific Standard Time)-CAPS.csv
lob_caps/Fri Nov 25 2022 07:50:10 GMT-0800 (Pacific Standard Time)-CAPS.csv
lob_caps/Thu Oct 27 2022 07:39:06 GMT-0700 (Pacific Daylight Time)-CAPS.csv
lob_caps/Wed Oct 12 2022 07:36:21 GMT-0700 (Pacific Daylight Time)-CAPS.csv
lob_caps/Thu Dec 01 2022 07:42:09 GMT-0800 (Pacific Standard Time)-CAPS.csv
lob_caps/Sat Oct 22 2022 08:57:53 GMT-0700 (Pacific Daylight Time)-CAPS.csv
lob_caps/Tue Sep 13 2022 06:57:20 GMT-0700 (Pacific Daylight Time)-CAPS.csv
lob_caps/Tue Oct 25 2022 06:50:17 GMT-0700 (Pacific Daylight Time)-CAPS.csv
lob_caps/Sat

In [23]:
def getIndicators():
    return pd.read_csv("usIndicatorsFall22-fulls.csv")

return a list of all indicator dates, as full readable dates

In [18]:
def getlob(csv):
        print("GETTING LOB FILE FOR: csv named ",csv)
        csv = "lob_caps/" + csv
        df = pd.read_csv(csv)
            # print("\tfor ", type, df.columns)
        return df

In [20]:
from dateutil import parser
import datetime
import time

def convertIndicatorDateToEpoch(original):
    # print("original ", original) #Thu Aug 11 4:30 AM 2022
    re =time.strptime(original, "%a %b %d %I:%M %p %Y")
    ep = time.mktime(re)
    # print("TS ", int(ep))   #1660311116579
    return int(ep)

In [25]:
def getFullIADates():
    data = getIndicators() #indicators file read, access all indicator data per event/announcement
    result = []
    for row, val in data.iterrows(): #all indicators
        s1 = val['date'].replace(','  , '')
        ss = s1.split(" ")
        day  = ss[0][0:3]
        mon = ss[1][0:3]
        num = ss[2]
        hr = val['time']
        if int(num) <10:
            num = "0"+num
        yr = ss[3]
        l = day + " " + mon + " " + num #+ " " + hr
        extendedDateTime = day + " " + mon + " " + num + " " + hr + " " +yr
        print("FOR INDICATOR, DATED: ",extendedDateTime)
        IaDict = { "dictDF":val, "epochForIA":convertIndicatorDateToEpoch( extendedDateTime)}
        result.append( IaDict) #returns df/caps and the date associated    
    return result

In [11]:
vizIAwithCaps = [] #a list of capitalization dataframes, with respecitve indicators

inds = getFullIADates()  #all dates for indicators, as epochs inds[i][dictDF] / epochForIA
for key, val in endFrame.iterrows():  #all caps files, endFrame possesses capitalization stats, bc/ac. vols
    print(int(val['time']))
    capsWithIndicatorDict = {}
    min = val['time'].min()
    max = val['time'].max() #respective range of dates, as epoch
    for i in range(len(inds)): #scan indicators for a date that falls inside the min/max range of time
        if inds[i]['epochForIA'] <= max and inds[i]['epochForIA'] <= min:
            print("Economic indicator for caps is ", inds[i]['dictDF']['indicator'])
            print("occuring at time ", inds[i]['dicForIA']['epochForIA'])
            capsWithIndicatorDict["indicatorDF"] = inds[i]['dictDF']   #indicator's data frame row (series?)
            capsWithIndicatorDict['indicatorEpoch'] = inds[i]['epochForIA']  #epoch for indicator, should it fall within the range of the csv's data 
            capsWithIndicatorDict['capsDF'] = val 
            vizIAwithCaps.append(capsWithIndicatorDict)
            


1661520441572
1661520445856
1661520448511
1661520450877
1661520453861
1661520457211
1661520460015
1661520462515
1661520465766
1661520468801
1661520471599
1661520474650
1661520477486
1661520480625
1661520483576
1661520486658
1661520489619
1661520492659
1661520495647
1661520498621
1661520501637
1661520504555
1661520507801
1661520510623
1661520513542
1661520516626
1661520519512
1661520522551
1661520525624
1661520528536
1661520531465
1661520534542
1661520537623
1661520540575
1661520543615
1661520546575
1661520549595
1661520552602
1661520555671
1661520558543
1661520562086
1661520564583
1661520567603
1661520570566
1661520573552
1661520576515
1661520579609
1661520582689
1661520586222
1661520588638
1661520591562
1661520594664
1661520597613
1661520600637
1661520603957
1661520606645
1661520609731
1661520612559
1661520615821
1661520618881
1661520622129
1661520625588
1661520628751
1661520631773
1661520634712
1661520637905
1661520641539
1661520644513
1661520648238
1661520651297
1661520654348
166152

In [24]:
import datetime
def demarcateByAnnounceTime(pkg):
    
    for i in range(len(dfArray)):
        if i > 9:
            if isinstance(dfArray[i], dict):
                # print(i, "iterated 1, ",  dfArray[i], "iterated 2, ", type(dfArray[i])) #every item is a dict
                
                print("start time: ", dfArray[i]["df"]["time"].min())
                print("end time: ", dfArray[i]["df"]["time"].max())
                print("date of data: ", dfArray[i]["date"], "\n")
                
                data1 = dfArray[i]["df"]['mp']
                data2 = dfArray[i]["df"]['bc']
                t= dfArray[i]["df"]['time']
                ai = dfArray[i]["time"]
                fig, ax1 = plt.subplots()
                tt = i + 1
                # if isinstance(dfArray[tt]["time"], str):
                #     indEpoch = convertIndicatorDateToEpoch(dfArray[tt]["time"])#.split(" ") # turn this to epoch Mon Aug 01 6:00 AM
                #     print("Announcement time: ",indEpoch)
                color = 'tab:red'
                ax1.set_xlabel('time (s)')
                ax1.set_ylabel('price, AVAX', color=color)
                ax1.plot(t, data1, color=color)
                ax1.tick_params(axis='y', labelcolor=color)

                ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

                color = 'tab:blue'
                ax2.set_ylabel('buy capitalization', color=color)  # we already handled the x-label with ax1
                ax2.plot(t, data2, color=color)
                ax2.tick_params(axis='y', labelcolor=color)

                fig.tight_layout()  # otherwise the right y-label is slightly clipped
                e = dfArray[i]["df"]['time'].loc[0]   #datetime.datetime.fromtimestamp(
                comb = e ," Buy Cap versus Price" #.strftime('%m-%d')
                plt.title(comb)
                #add vertical line at x=4
                # if indEpoch:
                #     plt.axvline(x=indEpoch, color='black', linestyle='-')
                plt.show()
            
        
        
        
demarcateByAnnounceTime( vizIAwithCaps)  #send the collected 
    

NameError: name 'getFullIADates' is not defined

In [ ]:
                                # print(type(inds), inds.columns)
def genDatesForIndicators():
    inds = getIndicators() 
    ret = []               
    for key, val in inds.iterrows():
                                    # print(key, val["date"], val["time"])
        s1 = val['date'].replace(','  , '')
        ss = s1.split(" ")
        day  = ss[0][0:3]
        mon = ss[1][0:3]
        num = ss[2]
        hr = val['time']
        if int(num) <10:
            num = "0"+num
        yr = ss[3]
        l = day + " " + mon + " " + num #+ " " + hr
        extendedDateTime = day + " " + mon + " " + num + " " + hr + " " +yr
                            # print("FOR INDICATOR, DATED: ",extendedDateTime)
        d = convertIndicatorDateToEpoch(extendedDateTime)
        # print(d)
        ret.append(np.int64(d))
    return ret
        
dateArr = genDatesForIndicators()
for i in range(len(dateArr)):
    print(type(dateArr[i]))

endFrame: collected time series, several months, 111K+ events

dateArr: list of all dates, as epochs for all indicators (dates only)

In [ ]:
l = endFrame[(endFrame['time'] <= 1668178800 ) & (endFrame['time'] > 1664578800   )] #1664578800  -->  1668178800
print(l.shape[0])

for key, val in endFrame.iterrows():
    

In [ ]:
# thresh = ((1000 * 60) * 60 ) * 1
# print("threshold minus gate: ", thresh)
# print(endFrame.dtypes)


iterate all anouncement times, associate 1,000 data points before then after each, then calculate

GRAVEYARD 

In [ ]:
# sample = endFrame[endFrame['time'].between(dateArr[i], lowEnd, inclusive="both")]
    # set = endFrame.loc[
    # sample1 = endFrame[(endFrame['time']> lowEnd)]
    # sample = endFrame[(endFrame['time']<= dateArr[i])]# ] #(endFrame['time']<= dateArr[i])]    & (endFrame['time']> lowEnd)
    # s =endFrame[(endFrame['time'] > lowEnd ) & (endFrame['time'] <= dateArr[i] )]
    # sample = endFrame.loc[s]
    # sample = endFrame.loc[set]
    # set = endFrame.query(endFrame['time'] >= lowEnd & endFrame['time'] <= dateArr[i]).index
    # sample = endFrame[set]
    # if int(sample.shape[0]) > 0:
    #  print(s.head()) #ok  
    # print(sample1.shape[0])
        # print(sample["time"].min(), " --> ", sample["time"].max(), "  --> ", sample.shape[0], " items")
    
    
        # sInd = endFrame.iloc[(endFrame['time']-dateArr[i]).abs().argsort()[:1]]#no
    # print("min matching time: ",sInd["time"])  #complete dataframe
    # endFrame[(endFrame['time']==)]
    
    
    
    #get moments 1 hours before the announcement
# print(endFrame["time"].min(), " --> ", endFrame["time"].max(), "  --> ", endFrame.shape[0], " items")

# import math
# for i in range(len(dateArr)): 
    # print("search for : ",dateArr[i])  #int 
# lowEnd = dateArr[i] - thresh  
# print(lowEnd, " --> ", dateArr[i])   #

# print(endFrame[endFrame['time'].isin(dateArr)])

        # print("compare: ", int(val['time']), dateArr[i])
        # if math.isclose(dateArr[i], val['time'], abs_tol = 15000):
        #     print("matched", val['time'])
            # sInd = endFrame.iloc[(endFrame['time']